# PyTorch 中的循环神经网络模块
前面我们讲了循环神经网络的基础知识和网络结构，下面我们教大家如何在 pytorch 下构建循环神经网络，因为 pytorch 的动态图机制，使得循环神经网络非常方便。

## 一般的 RNN

![](https://ws1.sinaimg.cn/large/006tKfTcly1fmt9xz889xj30kb07nglo.jpg)

对于最简单的 RNN，我们可以使用下面两种方式去调用，分别是 `torch.nn.RNNCell()` 和 `torch.nn.RNN()`，这两种方式的区别在于 `RNNCell()` 只能接受序列中单步的输入，且必须传入隐藏状态，而 `RNN()` 可以接受一个序列的输入，默认会传入全 0 的隐藏状态，也可以自己申明隐藏状态传入。

`RNN()` 里面的参数有

input_size 表示输入 $x_t$ 的特征维度

hidden_size 表示输出的特征维度

num_layers 表示网络的层数

nonlinearity 表示选用的非线性激活函数，默认是 'tanh'

bias 表示是否使用偏置，默认使用

batch_first 表示输入数据的形式，默认是 False，就是这样形式，(seq, batch, feature)，也就是将序列长度放在第一位，batch 放在第二位

dropout 表示是否在输出层应用 dropout

bidirectional 表示是否使用双向的 rnn，默认是 False

对于 `RNNCell()`，里面的参数就少很多，只有 input_size，hidden_size，bias 以及 nonlinearity

In [1]:
import torch
from torch.autograd import Variable
from torch import nn

In [2]:
# 定义一个单步的 rnn
rnn_single = nn.RNNCell(input_size=100, hidden_size=200)

In [3]:
# 访问其中的参数
rnn_single.weight_hh

Parameter containing:
tensor([[ 0.0017,  0.0554,  0.0242,  ...,  0.0342,  0.0341,  0.0270],
        [-0.0665,  0.0331,  0.0224,  ...,  0.0024, -0.0442, -0.0473],
        [ 0.0665, -0.0435, -0.0320,  ..., -0.0476, -0.0492,  0.0228],
        ...,
        [ 0.0216,  0.0379, -0.0256,  ...,  0.0039, -0.0501,  0.0259],
        [-0.0440,  0.0583, -0.0625,  ...,  0.0664, -0.0587,  0.0355],
        [ 0.0055, -0.0113,  0.0082,  ..., -0.0258,  0.0226, -0.0611]],
       requires_grad=True)

In [4]:
# 构造一个序列，长为 6，batch 是 5， 特征是 100
x = Variable(torch.randn(6, 5, 100)) # 这是 rnn 的输入格式

In [5]:
# 定义初始的记忆状态
h_t = Variable(torch.zeros(5, 200))

In [6]:
# 传入 rnn
out = []
for i in range(6): # 通过循环 6 次作用在整个序列上
    h_t = rnn_single(x[i], h_t)
    out.append(h_t)

In [7]:
h_t

tensor([[ 1.1192e-01, -6.1639e-01, -5.8073e-01, -5.7899e-01, -8.2451e-01,
         -3.1661e-01,  1.2890e-01, -2.1869e-01, -5.3029e-01, -6.9749e-01,
          2.8282e-02,  1.5608e-01,  1.5785e-01, -4.5671e-01, -2.5668e-01,
         -1.8031e-02, -6.1105e-03,  6.3793e-01,  1.0037e-01,  4.5417e-01,
          1.1225e-01, -8.4959e-02,  2.4809e-01, -3.2668e-01,  1.2171e-01,
         -4.9908e-01, -1.1184e-01, -4.8481e-01, -5.9690e-01, -4.1263e-01,
         -2.7141e-01, -6.4730e-01, -8.2121e-01, -7.0165e-01,  1.9123e-01,
         -3.9249e-01, -2.2135e-01,  4.2468e-02,  4.2555e-01, -3.4718e-01,
         -4.0830e-01, -1.7724e-01, -1.4192e-01, -3.8849e-01,  7.5876e-01,
          3.8622e-01,  5.1381e-01, -2.8480e-01,  7.2065e-01,  5.1339e-01,
         -3.5273e-01, -1.6775e-01, -3.6586e-01,  5.1995e-01, -2.4668e-01,
         -3.2339e-01,  3.4575e-01,  8.8290e-02,  4.3303e-01,  1.3298e-01,
          2.3943e-01, -1.6516e-01, -1.2847e-01, -5.4402e-01,  5.9719e-02,
         -1.4766e-01, -3.6279e-01, -5.

In [8]:
len(out)

6

In [9]:
out[0].shape # 每个输出的维度

torch.Size([5, 200])

可以看到经过了 rnn 之后，隐藏状态的值已经被改变了，因为网络记忆了序列中的信息，同时输出 6 个结果

下面我们看看直接使用 `RNN` 的情况

In [10]:
rnn_seq = nn.RNN(100, 200)

In [11]:
# 访问其中的参数
rnn_seq.weight_hh_l0

Parameter containing:
tensor([[-0.0315,  0.0560,  0.0674,  ...,  0.0069, -0.0687,  0.0104],
        [-0.0203,  0.0298,  0.0277,  ...,  0.0272, -0.0577, -0.0265],
        [-0.0102, -0.0562, -0.0475,  ...,  0.0496,  0.0538,  0.0678],
        ...,
        [ 0.0393, -0.0396, -0.0301,  ..., -0.0633,  0.0620, -0.0272],
        [ 0.0575, -0.0578,  0.0410,  ..., -0.0617, -0.0135, -0.0405],
        [-0.0355,  0.0674,  0.0393,  ..., -0.0422, -0.0124,  0.0188]],
       requires_grad=True)

In [12]:
out, h_t = rnn_seq(x) # 使用默认的全 0 隐藏状态

In [13]:
h_t

tensor([[[-0.6464,  0.4310,  0.1735, -0.0029, -0.2469,  0.2269,  0.5002,
          -0.2158, -0.1109,  0.1443, -0.4294, -0.4578, -0.0651, -0.1984,
          -0.1711,  0.6822,  0.5092, -0.2903,  0.2618,  0.3165,  0.7624,
          -0.0859, -0.6386, -0.0857, -0.5947, -0.0989, -0.4728, -0.2987,
           0.5189, -0.1217,  0.6292,  0.2503,  0.2950, -0.3765,  0.3072,
          -0.4154,  0.4455,  0.6234, -0.5041,  0.4064, -0.5308,  0.1635,
          -0.3283, -0.6610, -0.3698, -0.5952, -0.1402, -0.0320, -0.2006,
           0.5332, -0.3046,  0.5046, -0.7274, -0.0357,  0.0761,  0.1043,
          -0.0795, -0.0103,  0.3266,  0.4823,  0.0165, -0.5753, -0.3782,
           0.0457,  0.3205, -0.1039,  0.0166,  0.0594, -0.5656, -0.7366,
           0.6844, -0.6308,  0.6535,  0.0489, -0.7027,  0.4977,  0.1857,
           0.0876,  0.5076,  0.6454, -0.1615, -0.5660,  0.3423,  0.6580,
          -0.5908, -0.5660,  0.1213, -0.3928,  0.3552, -0.0388,  0.1078,
           0.4166,  0.6828,  0.1173,  0.1253,  0.13

In [14]:
len(out)

6

这里的 h_t 是网络最后的隐藏状态，网络也输出了 6 个结果

In [15]:
# 自己定义初始的隐藏状态
h_0 = Variable(torch.randn(1, 5, 200))

这里的隐藏状态的大小有三个维度，分别是 (num_layers * num_direction, batch, hidden_size)

In [16]:
out, h_t = rnn_seq(x, h_0)

In [17]:
h_t

tensor([[[-0.6530,  0.4270,  0.1720, -0.0401, -0.2526,  0.2363,  0.5059,
          -0.2099, -0.0789,  0.1200, -0.4395, -0.4470, -0.0761, -0.1699,
          -0.1844,  0.6854,  0.5073, -0.2933,  0.2867,  0.3108,  0.7582,
          -0.0975, -0.6470, -0.1226, -0.5883, -0.0863, -0.4717, -0.2995,
           0.5245, -0.1147,  0.6348,  0.2238,  0.2548, -0.3738,  0.2868,
          -0.4163,  0.4432,  0.6127, -0.5235,  0.3998, -0.5295,  0.1504,
          -0.3470, -0.6531, -0.3764, -0.6117, -0.1373, -0.0138, -0.2148,
           0.5217, -0.2893,  0.4849, -0.7287, -0.0340,  0.0677,  0.1018,
          -0.0823, -0.0073,  0.2973,  0.4971,  0.0230, -0.5788, -0.3707,
           0.0624,  0.2898, -0.1122,  0.0190,  0.0574, -0.5549, -0.7421,
           0.6917, -0.6362,  0.6582,  0.0281, -0.6959,  0.5071,  0.1721,
           0.0796,  0.5065,  0.6299, -0.1944, -0.5672,  0.3198,  0.6510,
          -0.6043, -0.5672,  0.1405, -0.3747,  0.3572, -0.0306,  0.1041,
           0.3969,  0.6796,  0.0980,  0.1213,  0.13

In [18]:
out.shape

torch.Size([6, 5, 200])

同时输出的结果也是 (seq, batch, feature)

一般情况下我们都是用 `nn.RNN()` 而不是 `nn.RNNCell()`，因为 `nn.RNN()` 能够避免我们手动写循环，非常方便，同时如果不特别说明，我们也会选择使用默认的全 0 初始化隐藏状态

## LSTM

![](https://ws1.sinaimg.cn/large/006tKfTcly1fmt9qj3uhmj30iz07ct90.jpg)

LSTM 和基本的 RNN 是一样的，他的参数也是相同的，同时他也有 `nn.LSTMCell()` 和 `nn.LSTM()` 两种形式，跟前面讲的都是相同的，我们就不再赘述了，下面直接举个小例子

In [19]:
lstm_seq = nn.LSTM(50, 100, num_layers=2) # 输入维度 100，输出 200，两层

In [20]:
lstm_seq.weight_hh_l0 # 第一层的 h_t 权重

Parameter containing:
tensor([[-0.0629,  0.0957, -0.0805,  ...,  0.0595,  0.0232,  0.0558],
        [ 0.0104, -0.0747,  0.0562,  ..., -0.0500, -0.0529, -0.0866],
        [-0.0272,  0.0296, -0.0420,  ...,  0.0223,  0.0097, -0.0639],
        ...,
        [-0.0229, -0.0430, -0.0777,  ..., -0.0386,  0.0085, -0.0534],
        [ 0.0433,  0.0155,  0.0221,  ...,  0.0278, -0.0140,  0.0900],
        [ 0.0838,  0.0498, -0.0683,  ..., -0.0560,  0.0345,  0.0475]],
       requires_grad=True)

**小练习：想想为什么这个系数的大小是 (400, 100)**

In [21]:
lstm_input = Variable(torch.randn(10, 3, 50)) # 序列 10，batch 是 3，输入维度 50

In [22]:
out, (h, c) = lstm_seq(lstm_input) # 使用默认的全 0 隐藏状态

注意这里 LSTM 输出的隐藏状态有两个，h 和 c，就是上图中的每个 cell 之间的两个箭头，这两个隐藏状态的大小都是相同的，(num_layers * direction, batch, feature)

In [23]:
h.shape # 两层，Batch 是 3，特征是 100

torch.Size([2, 3, 100])

In [24]:
c.shape

torch.Size([2, 3, 100])

In [25]:
out.shape

torch.Size([10, 3, 100])

我们可以不使用默认的隐藏状态，这是需要传入两个张量

In [26]:
h_init = Variable(torch.randn(2, 3, 100))
c_init = Variable(torch.randn(2, 3, 100))

In [27]:
out, (h, c) = lstm_seq(lstm_input, (h_init, c_init))

In [28]:
h.shape

torch.Size([2, 3, 100])

In [29]:
c.shape

torch.Size([2, 3, 100])

In [30]:
out.shape

torch.Size([10, 3, 100])

# GRU
![](https://ws3.sinaimg.cn/large/006tKfTcly1fmtaj38y9sj30io06bmxc.jpg)

GRU 和前面讲的这两个是同样的道理，就不再细说，还是演示一下例子

In [31]:
gru_seq = nn.GRU(10, 20)
gru_input = Variable(torch.randn(3, 32, 10))

out, h = gru_seq(gru_input)

In [32]:
gru_seq.weight_hh_l0

Parameter containing:
tensor([[ 0.2135, -0.1716,  0.1258,  ..., -0.0164, -0.1924, -0.0120],
        [-0.1850, -0.0314,  0.1661,  ...,  0.2139,  0.1155, -0.2230],
        [-0.0415,  0.1780, -0.1472,  ...,  0.1866, -0.0065, -0.1618],
        ...,
        [ 0.1897, -0.0523,  0.0345,  ..., -0.1825, -0.1864, -0.0457],
        [-0.0254,  0.1764,  0.0131,  ...,  0.1100, -0.1790, -0.1184],
        [-0.0352, -0.1037, -0.1676,  ..., -0.1750,  0.0903, -0.1860]],
       requires_grad=True)

In [33]:
h.shape

torch.Size([1, 32, 20])

In [34]:
out.shape

torch.Size([3, 32, 20])